### implementing keras blog to use pre trianed word embedding 

#### URL : https://blog.keras.io/using-pre-trained-word-embeddings-in-a-keras-model.html

In [16]:
path='./data/20_newsgroup/'
GLOVE_DIR='./data/'

In [17]:
MAX_NB_WORDS=20000
MAX_SEQUENCE_LENGTH=1000
VALIDATION_SPLIT=0.2
EMBEDDING_DIM=100

In [18]:
import os

In [19]:
texts=[]
labels=[]
label_index={}

for dir_name in sorted(os.listdir(path)):
    npath=os.path.join(path,dir_name)
    if os.path.isdir(npath):
        label_id=len(label_index)
        label_index[dir_name]=label_id
        for file_name in sorted(os.listdir(npath)):
            if file_name.isdigit():
                fpath=os.path.join(npath,file_name)
                f=open(fpath, encoding='latin-1')
                t=f.read()
                i=t.find('\n\n')
                if i>0:
                    t=t[i:]
                texts.append(t)
                f.close()
                labels.append(label_id)            

In [20]:
len(labels)

19997

In [21]:
len(texts)

19997

 ## preparing the data


In [22]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [23]:
tokenizer=Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(texts)
sequences=tokenizer.texts_to_sequences(texts)

In [24]:
word_index=tokenizer.word_index

In [25]:
len(word_index)

174074

In [26]:
sequences=pad_sequences(sequences,maxlen=MAX_SEQUENCE_LENGTH)

In [27]:
labels

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [28]:
from keras.utils import to_categorical
import numpy as np
labels = to_categorical(np.asarray(labels))

In [29]:
print(sequences.shape)
print(labels.shape)

(19997, 1000)
(19997, 20)


In [31]:
labels

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [14]:
indices=np.arange(sequences.shape[0])

In [15]:
np.random.shuffle(indices)

In [16]:
sequences=sequences[indices]

In [17]:
labels=labels[indices]

In [18]:
nb_validation_samples=int(VALIDATION_SPLIT * sequences.shape[0])

In [19]:
nb_validation_samples

3999

In [20]:
x_train=sequences[:-nb_validation_samples]
x_test=sequences[-nb_validation_samples:]
y_train=labels[:-nb_validation_samples]
y_test=labels[-nb_validation_samples:]

## preparing embedding layer

In [21]:
embedding_index={}

In [22]:
f=open(os.path.join(GLOVE_DIR,'glove.6B.100d.txt'))

In [23]:
for line in f:
    values=line.split()
    word=values[0]
    embeddings=np.asarray(values[1:],dtype='float32')
    embedding_index[word]=embeddings
f.close()

In [24]:
print(len(embedding_index))

400000


In [25]:
embedding_matrix=np.zeros((len(word_index)+1,EMBEDDING_DIM))

In [26]:
for word,i in word_index.items():
    embedding_vector=embedding_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i]=embedding_vector

## embedding layer

In [27]:
from keras.layers import Embedding, Input, Conv1D, MaxPooling1D, Flatten, Dense, GlobalMaxPooling1D
#Embedding?

In [28]:
from keras.layers import Embedding
from keras.models import Model
embedding_layer=Embedding(len(word_index)+1,EMBEDDING_DIM,weights=[embedding_matrix],input_length=MAX_SEQUENCE_LENGTH,trainable=False)

## Conv Net

In [29]:
sequence_input=Input(shape=(MAX_SEQUENCE_LENGTH,),dtype='int32')

In [30]:
embedded_sequences=embedding_layer(sequence_input)

In [31]:
x=Conv1D(128,5,activation='relu')(embedded_sequences)

In [32]:
x=MaxPooling1D(5)(x)

In [33]:
x=Conv1D(128,5,activation='relu')(x)

In [34]:
x=MaxPooling1D(5)(x)

In [35]:
x=Conv1D(128,5,activation='relu')(x)

In [36]:
x=GlobalMaxPooling1D()(x)

In [37]:
#x=Flatten()(x)

In [38]:
x=Dense(128, activation='relu')(x)

In [39]:
preds=Dense(len(label_index),activation='softmax')(x)

In [40]:
model=Model(sequence_input,preds)


In [41]:
model.compile(loss='categorical_crossentropy',optimizer='rmsprop',metrics=['acc'])

In [42]:
model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=10,batch_size=128)

Train on 15998 samples, validate on 3999 samples
Epoch 1/10
15998/15998 [==============================] - 251s 16ms/step - loss: 2.3917 - acc: 0.2177 - val_loss: 1.7935 - val_acc: 0.3571
Epoch 2/10
15998/15998 [==============================] - 249s 16ms/step - loss: 1.5407 - acc: 0.4649 - val_loss: 1.2983 - val_acc: 0.5524
Epoch 3/10
15998/15998 [==============================] - 249s 16ms/step - loss: 1.1959 - acc: 0.5819 - val_loss: 1.1080 - val_acc: 0.6147
Epoch 4/10
15998/15998 [==============================] - 244s 15ms/step - loss: 0.9889 - acc: 0.6586 - val_loss: 1.1178 - val_acc: 0.6182
Epoch 5/10
15998/15998 [==============================] - 243s 15ms/step - loss: 0.8382 - acc: 0.7184 - val_loss: 0.9485 - val_acc: 0.6759
Epoch 6/10
15998/15998 [==============================] - 242s 15ms/step - loss: 0.7077 - acc: 0.7606 - val_loss: 0.9329 - val_acc: 0.6859
Epoch 7/10
15998/15998 [==============================] - 1983s 124ms/step - loss: 0.6088 - acc: 0.7956 - val_loss: 0

In [43]:
model.save_weights('weights_pre_trained_word_embeddings.h5')

ImportError: `save_weights` requires h5py.